In [ ]:
pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np 
import spacy 
import en_core_web_sm 
from nltk.tag.stanford import StanfordNERTagger
from nltk.tokenize import word_tokenize 
from nltk.parse import CoreNLPParser
import json
import random
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score
from nltk.corpus.reader.conll import ConllCorpusReader
from keras.layers import merge, TimeDistributed, Dropout, Bidirectional, Flatten, Masking
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as f_score
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import random
import math
import time

In [ ]:
train = pd.read_csv("train.tsv", header=0, \
                    delimiter="\t", quoting=3)
test = pd.read_csv("test.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [ ]:
train.shape

(1316462, 1)

In [ ]:
test.shape

(1272841, 1)

In [ ]:
train.columns.values

In [ ]:
test.columns.values

array(['id,Doc_ID,Sent_ID,Word'], dtype=object)

In [ ]:
df = train

In [ ]:
print(df[:700])

      id  Doc_ID  Sent_ID               Word tag
0      1     1.0      1.0            Obesity   O
1      2     1.0      1.0                 in   O
2      3     1.0      1.0               Low-   O
3      4     1.0      1.0                and   O
4      5     1.0      1.0      Middle-Income   O
..   ...     ...      ...                ...  ..
695  696     5.0     32.0                  h   O
696  697     5.0     32.0  post-transfection   O
697  698     5.0     32.0                  .   O
698  699     5.0     33.0                 By   O
699  700     5.0     33.0                 72   O

[700 rows x 5 columns]


In [ ]:
df1 = test

In [ ]:
print(df1[:700])

          id   Doc_ID   Sent_ID    Word
0    4543834  30001.0  191283.0   CCCVA
1    4543835  30001.0  191283.0       ,
2    4543836  30001.0  191283.0  MANOVA
3    4543837  30001.0  191283.0       ,
4    4543838  30001.0  191283.0      my
..       ...      ...       ...     ...
695  4544529  30009.0  191317.0    Most
696  4544530  30009.0  191317.0       (
697  4544531  30009.0  191317.0    58.1
698  4544532  30009.0  191317.0       %
699  4544533  30009.0  191317.0       )

[700 rows x 4 columns]


In [ ]:
# Convert .tsv file to dataturks json format. 
import json
import logging
import sys
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

tsv_to_json_format("train.tsv",'train.json','abc')

ERROR:root:Unable to process file
error = [Errno 2] No such file or directory: 'Data/ner_corpus_260.tsv'
Traceback (most recent call last):
  File "<ipython-input-12-0aa3b8b9c7a1>", line 7, in tsv_to_json_format
    f=open(input_path,'r') # input file
FileNotFoundError: [Errno 2] No such file or directory: 'Data/ner_corpus_260.tsv'


In [ ]:
pip install –U spacy
python –m spacy download en

In [ ]:
spacy.prefer_gpu()
nlp = en_core_web_sm.load()

In [ ]:
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  

    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       


    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  
                    [annotations],  
                    drop=0.2,  
                    sgd=optimizer,  
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA, 20)

modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)


tp=0
tr=0
tf=0

ta=0
c=0    

for text,annot in df1:

    doc_to_test=prdnlp(text)

    d={}
    for ent in doc_to_test.ents:
        d[ent.label_]=[]
    for ent in doc_to_test.ents:
        d[ent.label_].append(ent.text)

    d={}

    for ent in doc_to_test.ents:
        d[ent.label_]=[0,0,0,0,0,0]
    for ent in doc_to_test.ents:
        doc_gold_text= prdnlp.make_doc(text)
        gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
        y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
        y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]  
        if(d[ent.label_][0]==0):
            print("For Entity "+ent.label_+"\n")   
            (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,average='weighted')
            a=accuracy_score(y_true,y_pred)
            d[ent.label_][0]=1
            d[ent.label_][1]+=p
            d[ent.label_][2]+=r
            d[ent.label_][3]+=f
            d[ent.label_][4]+=a
            d[ent.label_][5]+=1
    c+=1

In [ ]:
words = data["Word"].values.tolist()
tags = data["tag"].values.tolist()

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(iterator):
        optimizer.zero_grad()
        output = model(word, tag)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
N_EPOCHS = 50
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model-piyush.pt')
    
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} ')
    print(f'\t Val. Loss: {valid_loss:.3f} ')

 Epoch: 01 
Train Loss: 4.212
Val. Loss: 4.096 
Epoch: 02
Train Loss: 3.767
Val. Loss: 3.942 
Epoch: 03
Train Loss: 3.506
Val. Loss: 4.049
Epoch: 04
Train Loss: 3.351
Val. Loss: 3.907
Epoch: 05
Train Loss: 3.248
Val. Loss: 3.860
Epoch: 06
Train Loss: 3.144
Val. Loss: 3.795
Epoch: 07
Train Loss: 3.008
Val. Loss: 3.660
Epoch: 08
Train Loss: 2.856
Val. Loss: 3.582
Epoch: 09
Train Loss: 2.705
Val. Loss: 3.538
Epoch: 10
Train Loss: 2.559
Val. Loss: 3.397
Epoch: 11
Train Loss: 2.428
Val. Loss: 3.331
Epoch: 12
Train Loss: 2.275
Val. Loss: 3.290
Epoch: 13
Train Loss: 2.162
Val. Loss: 3.158
Epoch: 14
Train Loss: 2.038
Val. Loss: 3.117
Epoch: 15
Train Loss: 1.947
Val. Loss: 3.091
Epoch: 16
Train Loss: 1.850
Val. Loss: 3.062
Epoch: 17
Train Loss: 1.756
Val. Loss: 3.022
Epoch: 18 
Train Loss: 1.622
Val. Loss: 3.085 
Epoch: 19
Train Loss: 1.554
Val. Loss: 3.003
Epoch: 20
Train Loss: 1.480
Val. Loss: 3.043
Epoch: 21
Train Loss: 1.382
Val. Loss: 3.064
Epoch: 22
Train Loss: 1.303
Val. Loss: 3.040
Epoc

In [ ]:
model.load_state_dict(torch.load('tut1-model-piyush.pt'))
test_loss = evaluate(model, test_iterator, criterion)
print(f'| Test Loss: {test_loss:.3f} ')

Test Loss: 2.8


In [ ]:
pred = cross_val_predict(estimator=MemoryTagger(), X=words, y=tags, cv=5)

In [ ]:
report = classification_report(y_pred=pred, y_true=tags)
print(report)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

B-indications       0.68      0.37      0.48     18777
I-indications       0.58      0.37      0.45     15770
            O       0.99      1.00      0.99   1546622
          nan       0.00      0.00      0.00         1

     accuracy                           0.98   1581170
    macro avg       0.56      0.43      0.48   1581170
 weighted avg       0.98      0.98      0.98   1581170

